# Introduction

Rust has a number of features that allow you to manage your code’s organization, including which details are exposed, which details are private, and what names are in each scope in your programs. These features, sometimes collectively referred to as the module system, include:

- <span style="color:lightgreen">**Packages**: A Cargo feature that lets you build, test, and share crates</span>
- <span style="color:lightgreen">**Crates**: A tree of modules that produces a library or executable</span>
- <span style="color:lightgreen">**Modules and `use`**: Let you control the organization, scope, and privacy of paths</span>
- <span style="color:lightgreen">**Paths**: A way of naming an item, such as a struct, function, or module</span>


# Packages and Crates

*A crate is the smallest amount of code that the Rust compiler considers at a time, e.g. even a single `.rs` file*. Crates can contain modules, and the modules may be defined in other files that get compiled with the crate, as we’ll see in the coming sections. The **crate root** is a source file that the Rust compiler starts from and makes up the root module of your crate.

> **A crate can come in one of two forms: a binary crate or a library crate.** 
> - <span style="color:lightgreen">**Binary Crates**</span>: Programs with `main` and you can compile to an executable, such as a command-line program or a server and `src/main.rs` is the crate root 
> - <span style="color:lightgreen">**Library Crates**</span>: Don’t have a `main` function, and they don’t compile to an executable with `src/lib.rs` as the crate root. Instead, they define functionality intended to be shared with multiple projects, e.g. the `rand` crate

<span style="color:skyblue">*A **package** is a bundle of one or more crates that provides a set of functionality. A package contains a `Cargo.toml` file that describes how to build those crates*</span>. For example, `Cargo` is actually a package that contains the binary crate for the command-line tool we have been using to build our code. If a package contains `src/main.rs` and `src/lib.rs`, it has two crates: a binary and a library, both with the same name as the package. A package can have multiple binary crates by placing files in the `src/bin` directory: each file will be a separate binary crate.

# Defining Modules to Control Scope and Privacy

In this section, we’ll talk about 
- Modules and other parts of the module system, namely *paths* that allow you to name items
- The `use` keyword that brings a path into scope
- The `pub` keyword to make items public
- The `as` keyword, external packages, and the `glob` operator



## Modules Cheat Sheet
Say we have a `backyard` project with the following structure
```
backyard
├── Cargo.lock
├── Cargo.toml
└── src
    ├── garden
    │   └── vegetables.rs
    ├── garden.rs
    └── main.rs
```
1. **Start from the crate root**: When compiling a crate, the compiler first looks in the crate root file (usually `src/lib.rs` for a library crate or `src/main.rs` for a binary crate) for code to compile. <span style="color:lightgreen">*`src/main.rs` and `src/lib.rs` are called crate roots because the contents of either of these two files form a module named crate at the root of the crate’s module structure, known as the **module tree**.*</span>
2. **Declaring modules**: In the crate root file, you can declare new modules; say, you declare a “garden” module with `mod garden`;. The compiler will look for the module’s code in these places:
    - Inline, within curly brackets that replace the semicolon following `mod garden`
    - In the file `src/garden.rs`
    - In the file `src/garden/mod.rs`
3. **Declaring submodules**: In any file other than the crate root, you can declare submodules. For example, you might declare `mod vegetables`; in `src/garden.rs`. The compiler will look for the submodule’s code within the directory named for the parent module in these places:
    - Inline, directly following `mod vegetables`, within curly brackets instead of the semicolon
    - In the file `src/garden/vegetables.rs`
    - In the file `src/garden/vegetables/mod.rs`
4. **Paths to code in modules**: Once a module is part of your crate, you can refer to code in that module from anywhere else in that same crate, as long as the privacy rules allow, using the path to the code. For example, an Asparagus type in the garden vegetables module would be found at `crate::garden::vegetables::Asparagus`.
5. **Private vs public**: Code within a module is private from its parent modules by default. To make a module public, declare it with `pub mod` instead of `mod`. To make items within a public module public as well, use `pub` before their declarations.
6. **The `use` keyword**: Within a scope, the `use` keyword creates shortcuts to items to reduce repetition of long paths. In any scope that can refer to `crate::garden::vegetables::Asparagus`, you can create a shortcut with use `crate::garden::vegetables::Asparagus`; and from then on you only need to write `Asparagus` to make use of that type in the scope.

The final module tree of the `backyard` crate is 

<img src="../images/8.png" style="width: 30%;">

## Grouping Related Code in Modules

Modules let us organize code within a crate for readability and easy reuse. Modules also allow us to control the privacy of items, because code within a module is private by default.

For example, let's create a new library named `restaurant` by running `cargo new restaurant --lib`. Then in `lib.rs`, we do
```rust
// lib.rs
mod front_of_house {
    mod hosting {
        fn add_to_waitlist() {}
        fn seat_at_table() {}
    }

    mod serving {
        fn take_order() {}
        fn serve_order() {}
        fn take_payment() {}
    }
}
```

The module tree of this crate is (created with the `cargo-modules` crate)

<img src="../images/7.png" style="width: 30%;">

# Paths for Referring to an Item in the Module Tree

To show Rust where to find an item in a module tree, we use a path in the same way we use a path when navigating a filesystem. To call a function, we need to know its path.

A path can take two forms:
1. An absolute path is the full path starting from a crate root; for code from an external crate, the absolute path begins with the crate name, and for code from the current crate, it starts with the literal crate.
2. A relative path starts from the current module and uses self, super, or an identifier in the current module. Both absolute and relative paths are followed by one or more identifiers separated by double colons (`::`).

Example:

```rust
// lib.rs
mod front_of_house {
    mod hosting {
        fn add_to_waitlist() {}

        fn seat_at_table() {}
    }

    mod serving {
        fn take_order() {}

        fn serve_order() {}

        fn take_payment() {}
    }
}

pub fn eat_at_restaurant() {
    // Absolute path
    crate::front_of_house::hosting::add_to_waitlist();

    // Relative path
    front_of_house::hosting::add_to_waitlist();
}
```

*Using relative or absolute path depends on the situation, but our preference in general is to specify absolute paths because it’s more likely we’ll want to move code definitions and item calls independently of each other.*

<span style="color:orange">*Building the above code will give a `module 'hosting' is private` error since all items (functions, methods, structs, enums, modules, and constants) are private to parent modules by default. **In Rust, items in a parent module can’t use the private items inside child modules, but items in child modules can use the items in their ancestor modules**.*</span>

If we want the `eat_at_restaurant` function in the parent module to have access to the `add_to_waitlist` function in the child module, so we mark the hosting module with the `pub` keyword (note that we have to put `pub` both before the `hosting` module and the `add_to_waitlist` function, not only for `hosting`):
```rust
// lib.rs
mod front_of_house {
    pub mod hosting {
        pub fn add_to_waitlist() {}
    }
}

pub fn eat_at_restaurant() {
    // Absolute path
    crate::front_of_house::hosting::add_to_waitlist();

    // Relative path
    front_of_house::hosting::add_to_waitlist();
}
```

> **Packages with both `src/main.rs` and `src/lib.rs` crate roots**:  
> Typically, packages with this pattern of containing both a library and *a binary crate will have just enough code in the binary crate to start an executable that calls code with the library crate*

## Starting Relative Paths with `super`

`super` is like `..` in filesystem path

```rust 
fn deliver_order() {}

mod back_of_house {
    fn fix_incorrect_order() {
        cook_order();
        super::deliver_order();
    }

    fn cook_order() {}
}
```
The `fix_incorrect_order` function is in the `back_of_house` module, so we can use `super` to go to the parent module of `back_of_house`, which in this case is crate, the root


## Making Structs and Enums Public
### Structs
If we use `pub` before a struct definition, we make the struct public, but the struct’s fields will still be private (e.g. the `seasonal_fruit` of `Breakfast` below). We can make each field public or not on a case-by-case basis. We also need to make the `summer` public method for `Breakfast` that gives `seasonal_fruit` a value so we can construct an instance of `Breakfast` in `eat_at_restaurant`
```rust
pub struct Breakfast {
        pub toast: String,
        seasonal_fruit: String,
}

impl Breakfast {
        pub fn summer(toast: &str) -> Breakfast {
            Breakfast {
                toast: String::from(toast),
                seasonal_fruit: String::from("peaches"),
            }
        }
}

pub fn eat_at_restaurant() {
    // Order a breakfast in the summer with Rye toast
    let mut meal = back_of_house::Breakfast::summer("Rye");
    // Change our mind about what bread we'd like
    meal.toast = String::from("Wheat");
    println!("I'd like {} toast please", meal.toast);

    // The next line won't compile if we uncomment it; we're not allowed
    // to see or modify the seasonal fruit that comes with the meal
    // meal.seasonal_fruit = String::from("blueberries");
}
```

### Enum
In contrast, if we make an enum public, all of its variants are then public. We only need the `pub` before the `enum` keyword:
```rust
mod back_of_house {
    pub enum Appetizer {
        Soup,
        Salad,
    }
}

pub fn eat_at_restaurant() {
    let order1 = back_of_house::Appetizer::Soup;
    let order2 = back_of_house::Appetizer::Salad;
}
```

# Bringing Paths into Scope with the `use` Keyword
Instead of calling `crate::front_of_house::hosting::add_to_waitlist` all the time, we can do
```rust
use crate::front_of_house::hosting;

pub fn eat_at_restaurant() {
    hosting::add_to_waitlist();
}
```

Adding `use` and a path in a scope is similar to creating a symbolic link in the filesystem

Note that `use` only creates the shortcut for the particular scope in which the `use` occurs. For example, the below code won't work if we move `use crate::front_of_house::hosting;` out of the `customer` module
```rust
mod customer {
    use crate::front_of_house::hosting;

    pub fn eat_at_restaurant() {
        hosting::add_to_waitlist();
    }
}
```

### Creating Idiomatic `use` Paths

We can bring a function in an unidiomatic way like below
```rust
mod front_of_house {
    pub mod hosting {
        pub fn add_to_waitlist() {}
    }
}

use crate::front_of_house::hosting::add_to_waitlist;

pub fn eat_at_restaurant() {
    add_to_waitlist();
}
```
However, using unidiomatic way is not recommended, as bringing the function’s parent module into scope with use means we have to specify the parent module when calling the function (e.g. `hosting::add_to_waitlist`). <span style="color:lightgreen">*Specifying the parent module when calling the function makes it clear that the function isn’t locally defined while still minimizing repetition of the full path (idiomatic way)*</span>

On the other hand, when bringing in structs, enums, and other items with use, it’s idiomatic to specify the full path:
```rust
use std::collections::HashMap;

fn main() {
    let mut map = HashMap::new();
    map.insert(1, 2);
}
```

### Providing New Names with the `as` Keyword

If we bring 2 items with the same name into scope like below, we need to specify their parent modules
```rust
use std::fmt;
use std::io;

fn function1() -> fmt::Result {}

fn function2() -> io::Result<()> {}
```
Or we can use `as`
```rust
use std::fmt::Result;
use std::io::Result as IoResult;

fn function1() -> Result {}

fn function2() -> IoResult<()> {}
```



### Re-exporting Names with `pub use`
When we bring a name into scope with the `use` keyword, the name available in the new scope is private. To enable the code that calls our code to refer to that name as if it had been defined in that code’s scope, we can combine `pub` and `use`. This technique is called re-exporting because we’re bringing an item into scope but also making that item available for others to bring into their scope.

```rust
mod front_of_house {
    pub mod hosting {
        pub fn add_to_waitlist() {}
    }
}

pub use crate::front_of_house::hosting;

pub fn eat_at_restaurant() {
    hosting::add_to_waitlist();
}
```

Before this change, external code would have to call the add_to_waitlist function by using the path `restaurant::front_of_house::hosting::add_to_waitlist()`. Now that this `pub use` has re-exported the hosting module from the root module, external code can now use the path `restaurant::hosting::add_to_waitlist()` instead.



### Using External Packages
For example, using the `rand` external package:
1. Put `rand = "0.8.5"` to `Cargo.toml`
2. Bring `rand`'s `Rng` item into the scope with `use rand::Rng;`

Note that the standard `std` library is also a crate that’s external to our package. Because the standard library is shipped with the Rust language, we don’t need to change `Cargo.toml` to include `std`. But we do need to refer to it with use to bring items from there into our package’s scope.

### Using Nested Paths to Clean Up Large use Lists
Instead of doing
```rust
use std::cmp::Ordering;
use std::io;
```
we can combine them into one using nested paths
```rust
use std::{cmp::Ordering, io};
```

### The glob operator
If we want to bring all public items defined in a path into scope, we can specify that path followed by the `*` glob operator:
```rust
use std::collections::*;
```

# Separating Modules into Different Files

Let's split the code of `front_of_house` module into its own file (`src/front_of_house.rs`)

First, we declare that the `front_of_house` module whose body will be in `src/front_of_house.rs`
```rust
// src/lib.rs

mod front_of_house;

pub use crate::front_of_house::hosting;

pub fn eat_at_restaurant() {
    hosting::add_to_waitlist();
}
```

Then in `src/front_of_house.rs` we contain only the declaration of the `hosting` module
```rust
// src/front_of_house.rs
pub mod hosting;
```
Finally we create a `src/front_of_house` directory and a file `hosting.rs` to contain the definitions made in the hosting module
```rust
pub fn add_to_waitlist() {}
```

